In [1]:
import tensorflow as tf

#Import data

In [2]:
mnist=tf.keras.datasets.mnist
(X_train, y_train), (X_test, y_test)=mnist.load_data()

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [3]:
X_test = X_test/255.0
X_train = X_train/255.0

#Model

The Keras Sequential model is a simple, linear stack of layers in Keras, which is a popular deep learning API. It allows you to build neural networks layer by layer, where each layer has exactly one input and one output.

In [4]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  #This layer has 128 neurons, then it applies an activation function (ReLu in this case)
  tf.keras.layers.Dense(128, activation='relu'),
  #Dropout is a technique used to prevent overfitting in neural networks. During each training step, it randomly "drops out" (sets to 0) 20% of the neurons in the layer.
  tf.keras.layers.Dropout(0.2),
  #The output layer : 10 because we have 10 classes to predict from
  tf.keras.layers.Dense(10)
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [6]:
predictions = model(X_train[:1]).numpy()
predictions

array([[ 0.54695123,  0.17712364,  0.45484823,  0.07930426, -0.28771633,
         0.7759199 ,  0.22101998, -0.99081564,  0.01377916,  0.505365  ]],
      dtype=float32)

In [7]:
# Retourne les probas d'appartennaces a chaque classe
tf.nn.softmax(predictions).numpy()


array([[0.1350734 , 0.09331592, 0.12318844, 0.08462006, 0.05862438,
        0.16982813, 0.09750337, 0.02902192, 0.07925307, 0.1295714 ]],
      dtype=float32)

The loss is zero if the model is confident about the correct class. But the predictions made here are not meaningful because the model has not been trained. It will return random values based on the initial weights. To obtain valid predictions, you need to train the model on data.

In [8]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
loss_fn(y_train[:1], predictions).numpy()


1.7729684

#Model training

In [9]:
model.compile(
         # The optimizer is an algo that allows weights update while training.
              optimizer='adam',
         # Loss function
              loss=loss_fn,
              metrics=['accuracy'])

In [10]:
# Ajusting parameters of the model
model.fit(X_train, y_train, epochs=5)

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 12s 6ms/step - accuracy: 0.8586 - loss: 0.4832
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9552 - loss: 0.1528
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - accuracy: 0.9654 - loss: 0.1148
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9718 - loss: 0.0896
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.9767 - loss: 0.0762


**Difference between epochs and iterations :**
Let’s say you have a dataset with 1000 samples and you set the batch_size to 100.

Epoch: One complete pass through all 1000 samples.
Iterations: Since the batch size is 100, it will take 10 iterations (1000 / 100) to complete 1 epoch. After every iteration, the model’s weights are updated.

In [12]:
model.evaluate(X_test,  y_test, verbose=2) #Returns loss and accuracy

313/313 - 1s - 2ms/step - accuracy: 0.9766 - loss: 0.0776


[0.0776122435927391, 0.9765999913215637]

In [13]:
# Same model but we add a softmax layer to it to make it able to return probabilities
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [15]:
probability_model(X_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[5.1191018e-06, 1.3491378e-08, 4.8943848e-06, 1.5266981e-03,
        6.0733640e-10, 1.1415008e-06, 1.0037797e-13, 9.9844748e-01,
        7.5647085e-07, 1.3911845e-05],
       [5.1963458e-08, 1.4870960e-03, 9.9828583e-01, 1.9481358e-04,
        2.6443594e-14, 2.3540128e-05, 4.1772154e-07, 1.0877680e-12,
        8.2067772e-06, 3.7662308e-13],
       [9.9233041e-08, 9.9934655e-01, 9.0517664e-05, 4.1963012e-06,
        5.6531502e-05, 3.4898414e-06, 2.4719700e-06, 4.5243729e-04,
        4.3605287e-05, 1.7468305e-07],
       [9.9987388e-01, 1.8920880e-09, 2.9391162e-05, 3.8466692e-06,
        6.2779043e-07, 2.9089397e-06, 3.0362768e-05, 7.9370585e-07,
        8.0389702e-07, 5.7513771e-05],
       [1.0086075e-05, 2.3529747e-09, 1.6858688e-06, 3.5079839e-08,
        9.9133134e-01, 2.4667349e-06, 1.0410907e-05, 2.9065755e-05,
        4.1415376e-07, 8.6145522e-03]], dtype=float32)>